In [ ]:
import requests
from telegram import Bot, Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import CommandHandler, JobQueue, Application, CallbackQueryHandler, ContextTypes, CommandHandler, Updater
import datetime
import pandas as pd
import io

In [ ]:
TELEGRAM_TOKEN = "6649758324:AAGNvtZ6e4CTaEPwACa9o3IzUcXzmN7zZz0"
CHAT_ID = "@break_vol_binance"


bot = Bot(token=TELEGRAM_TOKEN)
await bot.initialize()

BASE_URL = "https://api.binance.com/api/v3"


In [ ]:
def get_symbol_data(symbol):
    url = f"{BASE_URL}/klines"
    params = {
        "symbol": symbol,
        "interval": "1h",
        "limit": 1
    }
    response = requests.get(url, params=params)
    data = response.json()
    columns = [
        "Kline open time",
        "Open price",
        "High price",
        "Low price",
        "Close price",
        "Volume",
        "Kline Close time",
        "Quote asset volume",
        "Number of trades",
        "Taker buy base asset volume",
        "Taker buy quote asset volume",
        "Unused field"
    ]
    df = pd.DataFrame(data, columns=columns)
    return df

def get_usdt_trading_pairs():
    response = requests.get("https://api.binance.com/api/v3/exchangeInfo")
    data = response.json()

    usdt_pairs = []

    for symbol_info in data['symbols']:
        if symbol_info['quoteAsset'] == 'USDT':
            usdt_pairs.append(symbol_info['symbol'])

    return usdt_pairs

# get MA20 volume
def get_ma20_volume(symbol):
    url = f"{BASE_URL}/klines"
    params = {
        "symbol": symbol,
        "interval": "1h",
        "limit": 20
    }
    response = requests.get(url, params=params)
    data = response.json()
    volumes = [float(entry[5]) for entry in data]
    ma20_volume = sum(volumes) / len(volumes)
    return ma20_volume

async def send_chart(chart_symbol, caption):
  print("Sending chart...")
#   query = update.callback_query
#   await query.answer()
  symbol = f"BINANCE:{chart_symbol}"
  # Get chart image using CHART-IMG API
  chart_api_key = "esFwC38dqg3anwgXuhbaB8UeGt2YWKHW7Lx1nm6Q"
  chart_url = "https://api.chart-img.com/v2/tradingview/advanced-chart"
  headers = {
  "x-api-key": chart_api_key,
  "content-type": "application/json"
  }
  chart_payload = {
  "theme": "dark",
  "interval": "1h",
  "symbol": symbol,
  "override": {
      "showStudyLastValue": True
  },
  "studies": [
      {
      "name": "Volume",
      "forceOverlay": False
      },
      {
        "name": "Ichimoku Cloud",
        "input": {
          "in_0": 9,
          "in_1": 26,
          "in_2": 52,
          "in_3": 26
        },
        "override": {
          "ConversionLine.visible": True,
          "ConversionLine.linewidth": 1,
          "ConversionLine.plottype": "line",
          "ConversionLine.color": "rgb(33,150,243)",
          "BaseLine.visible": True,
          "BaseLine.linewidth": 1,
          "BaseLine.plottype": "line",
          "BaseLine.color": "rgb(128,25,34)",
          "LaggingSpan.visible": True,
          "LaggingSpan.linewidth": 1,
          "LaggingSpan.plottype": "line",
          "LaggingSpan.color": "rgb(67,160,71)",
          "LeadingSpanA.visible": True,
          "LeadingSpanA.linewidth": 1,
          "LeadingSpanA.plottype": "line",
          "LeadingSpanA.color": "rgb(165,214,167)",
          "LeadingSpanB.visible": True,
          "LeadingSpanB.linewidth": 1,
          "LeadingSpanB.plottype": "line",
          "LeadingSpanB.color": "rgb(250,161,164)",
          "Plots Background.visible": True,
          "Plots Background.transparency": 90
        }
      }
  ]
  }
  chart_response = requests.post(chart_url, headers=headers, json=chart_payload)
  image_buffer = io.BytesIO(chart_response.content)

  await bot.send_photo(chat_id= CHAT_ID,photo=image_buffer, caption=caption)


# Check trading conditions and send message to Telegram
tokens_to_check = ["BTCUSDT", "UNIUSDT", "MAVUSDT", "OPUSDT","UNFIUSDT","DOGEUSDT",
                    "SUIUSDT","BAKEUSDT","AUDIOUSDT","GRTUSDT","LITUSDT","BELUSDT","RENUSDT"]
async def check_conditions_and_send_message(context):
    print("Checking conditions...")
    for symbol in tokens_to_check:
        symbol_data = get_symbol_data(symbol)
        current_price = float(symbol_data['Close price'])
        open_price = float(symbol_data["Open price"])
        current_volume = float(symbol_data["Volume"])
        ma20_volume = get_ma20_volume(symbol)

        if current_price > open_price and current_volume > ma20_volume:
            keyboard = [[InlineKeyboardButton(text=f"View Chart {symbol}", callback_data=symbol)]]
            reply_markup = InlineKeyboardMarkup(keyboard)
            message = f"🚀BUY SIGNAL {symbol} 📈\n💸Price: {current_price}\nOpen Price: {open_price}\nVolume: {current_volume}\nMA20 Volume: {ma20_volume}"
            # await bot.send_message(chat_id=CHAT_ID, text=message, reply_markup=reply_markup)
            await send_chart(symbol, message)


In [ ]:
current_time = datetime.datetime.now()
next_hour = current_time.replace(minute=0, second=0, microsecond=0) + datetime.timedelta(minutes=45)
time_to_wait = (next_hour - current_time).total_seconds()
print("Waiting for next hour to start. Time to wait: ", time_to_wait)

In [ ]:
application = Application.builder().token(TELEGRAM_TOKEN).build()
application.job_queue.run_repeating(check_conditions_and_send_message, interval=3600 , first= time_to_wait)
# application.add_handler(CommandHandler('start_check', check_conditions_and_send_message))
# application.add_handler(CallbackQueryHandler(send_chart))

In [ ]:
await application.initialize()
await application.start()
# await application.run_polling(allowed_updates=Update.ALL_TYPES)